In [ ]:
import pandas as pd
import re
import os
import uuid
import csv
import xml.etree.ElementTree as ET
from datetime import datetime

# === Папки ===
input_root = "wildberries"
output_csv = "final_csv"
output_xlsx = "final_xlsx"
output_xml = "final_xml"
output_orphan = "final_orphan"

for folder in [output_csv, output_xlsx, output_xml, output_orphan]:
    os.makedirs(folder, exist_ok=True)

# === Файлы справочников ===
database_file = "database.csv"
buyers_file = "database_buyers.csv"
seller_file = "database_seller.csv"

# === Загрузка справочников ===
db = pd.read_csv(database_file, sep=";", dtype=str).fillna("")
db.columns = db.columns.str.lower()

buyers = pd.read_csv(buyers_file, sep=";", dtype=str, index_col=None).fillna("")
buyers = buyers.reset_index(drop=True)
buyers.columns = buyers.columns.str.strip()

print("📌 Загруженные колонки database_buyers:", buyers.columns.tolist())
print("📄 Первые строки database_buyers:")
print(buyers.head())

seller_df = pd.read_csv(seller_file, sep=";", dtype=str).fillna("")
if seller_df.shape[0] == 0:
    raise RuntimeError("Файл seller_file пуст или не найден")
seller = seller_df.iloc[0].to_dict()
# нормализуем ключи продавца в нижний регистр (удобно потом обращаться)
seller = {k.lower(): ("" if pd.isna(v) else str(v)) for k, v in seller.items()}

print("📌 Загруженные колонки database_seller:", list(seller.keys()))
print("📄 Первая строка database_seller:")
print(seller)


# ===== Утилитарки =====
def extract_gtin(kiz: str):
    if not isinstance(kiz, str):
        return None
    clean = kiz.strip().strip('"').strip("'")
    m = re.match(r"010(\d{13})", clean)
    return m.group(1) if m else None

def safe_get(d: dict, key: str):
    if not d:
        return ""
    return str(d.get(key) or d.get(key.lower()) or d.get(key.upper()) or "")

def normalize_edo_id(s: str) -> str:
    """Гарантируем, что EDO id содержит только допустимые символы.
       Сохраняем дефисы '-' (они допустимы) и подчёркивания.
       Остальные символы заменяем на подчёркивание.
    """
    if s is None:
        return ""
    s = str(s).strip()
    # допустимые: буквы, цифры, дефис, подчёркивание
    return re.sub(r"[^A-Za-z0-9\-_]", "_", s)


def add_address(parent, row):
    """
    Добавляет <АдрРФ .../> в элемент parent.
    row может быть dict или pandas.Series — используем safe_get.
    """
    addr = {}
    if safe_get(row, "indeks"):
        addr["Индекс"] = safe_get(row, "indeks")
    if safe_get(row, "kodregion"):
        addr["КодРегион"] = safe_get(row, "kodregion")
    if safe_get(row, "naimregion"):
        addr["НаимРегион"] = safe_get(row, "naimregion")

    # город/район/нас.пункт: если есть город — ставим город,
    # иначе ставим Район+НаселПункт (если есть)
    gorod = safe_get(row, "gorod")
    rayon = safe_get(row, "raion") or safe_get(row, "rayon")  # допустим варианты
    nasel = safe_get(row, "naselpunk") or safe_get(row, "naselpunkt") or safe_get(row, "nasel_punkt")

    if gorod:
        addr["Город"] = gorod
    else:
        if rayon:
            addr["Район"] = rayon
        if nasel:
            addr["НаселПункт"] = nasel

    if safe_get(row, "ulitsa"):
        addr["Улица"] = safe_get(row, "ulitsa")
    if safe_get(row, "dom"):
        addr["Дом"] = safe_get(row, "dom")
    if safe_get(row, "kvart"):
        addr["Кварт"] = safe_get(row, "kvart")
    
    #RTZ popravka
    if safe_get(row, "korp"):
        addr["Корпус"] = safe_get(row, "kvart")


    # удаляем пустые ключи
    clean = {k: v for k, v in addr.items() if v is not None and v != ""}
    ET.SubElement(parent, "АдрРФ", clean)


# ===== XML генерация и сохранение =====
def create_xml(file, buyer_raw, records_df, seller):
    # приводим buyer к dict с нижними ключами
    buyer = {k.lower(): ("" if pd.isna(v) else str(v)) for k, v in buyer_raw.items()}

    today = datetime.today().strftime("%d.%m.%Y")
    base_name = os.path.splitext(os.path.basename(file))[0]

    # Дата отгрузки = из названия файла (если валидная)
    try:
        ship_date = datetime.strptime(base_name, "%d.%m.%Y").strftime("%d.%m.%Y")
    except Exception:
        ship_date = today

    # Формируем ИдФайл — строго по образцу (с нормализацией EDO id)
    ship_date_ymd = datetime.strptime(ship_date, "%d.%m.%Y").strftime("%Y%m%d")
    unique_id = str(uuid.uuid4())  # полный UUID в формате 8-4-4-4-12
    suffix = "0_1_0_0_0_00"

    # Берём EDO идентификаторы из полей 'id' (если есть) — не менять дефисы!
    seller_id_raw = seller.get("id") or seller.get("ID") or seller.get("inn") or seller.get("ogrnip") or ""
    buyer_id_raw = buyer.get("id") or buyer.get("ID") or buyer.get("inn") or buyer.get("ogrnip") or ""

    seller_id = normalize_edo_id(seller_id_raw)
    buyer_id = normalize_edo_id(buyer_id_raw)

    id_file = f"ON_NSCHFDOPPR_{buyer_id}_{seller_id}_{ship_date_ymd}_{unique_id}_{suffix}"
    out_file = os.path.join(output_xml, f"{id_file}.xml")


    # — построение XML
    root = ET.Element("Файл", {
        "ИдФайл": id_file,
        "ВерсФорм": "5.03",
        "ВерсПрог": "KIZProcessor 1.0"
    })

    doc = ET.SubElement(root, "Документ", {
        "КНД": "1115131",
        "Функция": "СЧФДОП",
        "ПоФактХЖ": "Документ об отгрузке товаров (выполнении работ), передаче имущественных прав (документ об оказании услуг)",
        "НаимДокОпр": "Счет-фактура и документ об отгрузке товаров (выполнении работ), передаче имущественных прав (документ об оказании услуг)",
        "ДатаИнфПр": today,
        "ВремИнфПр": datetime.now().strftime("%H.%M.%S")
    })

    # СвСчФакт
    sf = ET.SubElement(doc, "СвСчФакт", {"НомерДок": id_file, "ДатаДок": today})

    # Продавец
    svprod = ET.SubElement(sf, "СвПрод")
    idseller = ET.SubElement(svprod, "ИдСв")
    svip = ET.SubElement(idseller, "СвИП", {
        "ИННФЛ": safe_get(seller, "inn"),
        "ОГРНИП": safe_get(seller, "ogrnip")
    })
    ET.SubElement(svip, "ФИО", {
        "Фамилия": safe_get(seller, "familiya"),
        "Имя": safe_get(seller, "imya"),
        "Отчество": safe_get(seller, "otchestvo")
    })
    adr = ET.SubElement(svprod, "Адрес")
    add_address(adr, seller)

    # Покупатель
    svpokup = ET.SubElement(sf, "СвПокуп")
    idbuyer = ET.SubElement(svpokup, "ИдСв")
    svip_b = ET.SubElement(idbuyer, "СвИП", {
        "ИННФЛ": safe_get(buyer, "inn"),
        "ОГРНИП": safe_get(buyer, "ogrnip")
    })
    ET.SubElement(svip_b, "ФИО", {
        "Фамилия": safe_get(buyer, "familiya"),
        "Имя": safe_get(buyer, "imya"),
        "Отчество": safe_get(buyer, "otchestvo")
    })
    adr_b = ET.SubElement(svpokup, "Адрес")
    add_address(adr_b, buyer)

    ET.SubElement(sf, "ДенИзм", {"КодОКВ": "643", "НаимОКВ": "Российский рубль"})

    # Таблица товаров (сгруппировано по группе/price/okei/tax)
    tab = ET.SubElement(doc, "ТаблСчФакт")
    row_num = 1
    total_sum = 0.0
    for (group, price, okei, tax), subset in records_df.groupby(["group", "price", "okei", "tax"]):
        qty = len(subset)
        try:
            price_f = float(price)
        except Exception:
            price_f = 0.0
        sum_ = price_f * qty
        total_sum += sum_

        st = ET.SubElement(tab, "СведТов", {
            "НомСтр": str(row_num),
            "НаимТов": str(group),
            "ОКЕИ_Тов": str(okei),
            "НаимЕдИзм": "Штука",
            "КолТов": str(qty),
            "ЦенаТов": str(price),
            "СтТовБезНДС": str(sum_),
            "НалСт": str(tax),
            "СтТовУчНал": str(sum_)
        })

        dop = ET.SubElement(st, "ДопСведТов")
        nom = ET.SubElement(dop, "НомСредИдентТов")
        for kiz in subset["val"]:
            # Текст будет автоматически экранирован при записи
            ET.SubElement(nom, "КИЗ").text = str(kiz)

        ET.SubElement(st, "Акциз")
        ET.SubElement(st.find("Акциз"), "БезАкциз").text = "без акциза"
        sn = ET.SubElement(st, "СумНал")
        ET.SubElement(sn, "БезНДС").text = "без НДС"

        row_num += 1

    vs = ET.SubElement(tab, "ВсегоОпл", {
        "СтТовБезНДСВсего": str(total_sum),
        "СтТовУчНалВсего": str(total_sum)
    })
    snv = ET.SubElement(vs, "СумНалВсего")
    ET.SubElement(snv, "БезНДС").text = "без НДС"

    # ПродПер
    svpp = ET.SubElement(doc, "СвПродПер")
    swper = ET.SubElement(svpp, "СвПер", {"СодОпер": "Товары переданы", "ДатаПер": ship_date})
    ET.SubElement(swper, "БезДокОснПер").text = "1"

    # Подписант (заполняем ФИО продавца)
    pod = ET.SubElement(doc, "Подписант", {"Должн": "Индивидуальный Предприниматель", "СпосПодтПолном": "1"})
    ET.SubElement(pod, "ФИО", {
        "Фамилия": safe_get(seller, "familiya"),
        "Имя": safe_get(seller, "imya"),
        "Отчество": safe_get(seller, "otchestvo")
    })

    # === Сохраняем XML (имя файла == ИдФайл) ===
    tree = ET.ElementTree(root)
    tree.write(out_file, encoding="windows-1251", xml_declaration=True)

    # Контроль: имя файла (без .xml) должно точно совпадать с ИдФайл внутри
    filename_without_ext = os.path.splitext(os.path.basename(out_file))[0]
    xml_root = ET.parse(out_file).getroot()
    xml_idfile = xml_root.get("ИдФайл")
    print(f"Сохранён XML: {out_file}")
    if filename_without_ext != xml_idfile:
        raise RuntimeError(f"Несовпадение: имя файла '{filename_without_ext}' != ИдФайл в XML '{xml_idfile}'")
    else:
        print("✔ Имя файла соответствует атрибуту ИдФайл (всё ок).")


# ===== Обработка отдельного CSV =====
def process_file(file, db, buyer_row):
    print(f"🔎 Обработка файла: {file}")
    first_row = None
    with open(file, newline='', encoding='utf-8') as fh:
        rdr = csv.reader(fh)
        for r in rdr:
            if len(r) == 0:
                continue
            first_row = r
            break
    if first_row is None:
        print(f"⚠️ Файл пуст или не содержит строк: {file}")
        return

    kiz_row = [v for v in first_row if isinstance(v, str) and v.strip().startswith("010")]
    print(f"📦 Найдено КИЗ: {len(kiz_row)} (первые: {kiz_row[:5]})")

    records, orphans = [], []
    for val in kiz_row:
        gtin = extract_gtin(val)
        if gtin:
            matched = db.loc[db["gtin"] == gtin]
            if not matched.empty:
                r = matched.iloc[0]
                records.append({
                    "val": val,
                    "gtin": gtin,
                    "group": r.get("group", ""),
                    "price": r.get("price", "0"),
                    "okei": r.get("okei", ""),
                    "tax": r.get("tax", "")
                })
            else:
                orphans.append((val, gtin))
        else:
            orphans.append((val, ""))

    print(f"✅ Совпадений: {len(records)}, ❌ Сирот: {len(orphans)}")

    if orphans:
        base_name = os.path.splitext(os.path.basename(file))[0]
        orphan_file = os.path.join(output_orphan, f"{base_name}_orphan.csv")
        with open(orphan_file, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f, delimiter=";", quotechar='"', quoting=csv.QUOTE_MINIMAL)
            w.writerow(["КИЗ", "GTIN"])
            for o in orphans:
                w.writerow([o[0], o[1]])
        print(f"⚠️ Orphans saved: {orphan_file}")

    if not records:
        print(f"⚠️ Нет сопоставленных КИЗ в {file} — XML не формируется")
        return

    records_df = pd.DataFrame(records)
    create_xml(file, buyer_row.to_dict(), records_df, seller)


# ===== Запуск по всем покупателям/папкам =====
processed_files = 0
for i in range(len(buyers)):
    buyer_id = buyers.loc[i, "BuyerID"]
    buyer_folder = str(buyers.loc[i, "BuyerFolder"])
    folder = os.path.join(input_root, buyer_folder)

    print(f"➡️ BuyerID={buyer_id}, BuyerFolder={buyer_folder}, путь={folder}")

    if not os.path.exists(folder):
        print(f"⛔ Папка не найдена: {folder}")
        continue

    buyer_row = buyers.loc[i]
    for fname in os.listdir(folder):
        if not fname.lower().endswith(".csv"):
            continue
        fpath = os.path.join(folder, fname)
        process_file(fpath, db, buyer_row)
        processed_files += 1

print("🎯 Обработка завершена")
print(f"Files processed: {processed_files}")
